In [1]:
import h5py
print("h5py is installed. Version:", h5py.__version__)


ImportError: DLL load failed while importing _errors: The specified procedure could not be found.

In [2]:
!pip show h5py

Name: h5py
Version: 3.12.1
Summary: Read and write HDF5 files from Python
Home-page: 
Author: 
Author-email: Andrew Collette <andrew.collette@gmail.com>
License: BSD-3-Clause
Location: D:\Anaconda\Lib\site-packages
Requires: numpy
Required-by: keras, tensorflow-intel


In [3]:
!pip h5py

ERROR: unknown command "h5py"



In [4]:
!pip uninstall -y h5py
!pip install h5py --no-cache-dir

Found existing installation: h5py 3.12.1
Uninstalling h5py-3.12.1:
  Successfully uninstalled h5py-3.12.1


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_excel("mental_health_chatbot_dataset.xlsx")

df = df.dropna(subset=["User Input", "Bot Response"])
user_inputs = df["User Input"].astype(str).tolist()
bot_responses = df["Bot Response"].astype(str).tolist()

vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(user_inputs + bot_responses)

input_sequences = tokenizer.texts_to_sequences(user_inputs)
output_sequences = tokenizer.texts_to_sequences(bot_responses)

max_len = max(max(len(seq) for seq in input_sequences),
              max(len(seq) for seq in output_sequences))

input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding="post")
output_sequences = pad_sequences(output_sequences, maxlen=max_len, padding="post")

print("Tokenizer trained")
print("Vocab size:", len(tokenizer.word_index))
print("Max sequence length:", max_len)

Tokenizer trained
Vocab size: 210
Max sequence length: 16


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
integer_encoded_responses = label_encoder.fit_transform(bot_responses)

onehot_responses = to_categorical(integer_encoded_responses)

X_train, X_test, y_train, y_test = train_test_split(
    input_sequences, onehot_responses, test_size=0.2, random_state=42
)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))
print("Number of output classes:", onehot_responses.shape[1])

Training set size: 24
Test set size: 6
Number of output classes: 30


In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
import h5py

model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X_train.shape[1]),  
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dropout(0.3), 
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history = model.fit(
    X_train, y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    batch_size=32,
    callbacks=callbacks
)
model.save('mental_health_chatbot_model.keras')

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.0417 - loss: 3.4022 - val_accuracy: 0.0000e+00 - val_loss: 3.4069 - learning_rate: 0.0010
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0417 - loss: 3.3978 - val_accuracy: 0.0000e+00 - val_loss: 3.4176 - learning_rate: 0.0010
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.0417 - loss: 3.3881 - val_accuracy: 0.0000e+00 - val_loss: 3.4307 - learning_rate: 0.0010
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0417 - loss: 3.3857
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.0417 - loss: 3.3857 - val_accuracy: 0.0000e+00 - val_loss: 3.4484 - learning_rate: 0.0010
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.0417 - loss: 3.3819 - val_accuracy: 0.0000e+00 - val_loss: 3.4593 - learning_rate: 5.0000e-04
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.0417 - loss:

AttributeError: module 'h5py' has no attribute 'File'

In [1]:
from tensorflow.keras.models import load_model

model = load_model('mental_health_chatbot.keras')

In [13]:
!pip uninstall -y h5py

Found existing installation: h5py 3.13.0
Uninstalling h5py-3.13.0:
  Successfully uninstalled h5py-3.13.0


In [14]:
!pip install --no-cache-dir h5py

  Obtaining dependency information for h5py from https://files.pythonhosted.org/packages/56/89/e3ff23e07131ff73a72a349be9639e4de84e163af89c1c218b939459a98a/h5py-3.13.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
    --------------------------------------- 0.1/3.0 MB 409.6 kB/s eta 0:00:08
    --------------------------------------- 0.1/3.0 MB 357.2 kB/s eta 0:00:09
   -- ------------------------------------- 0.2/3.0 MB 657.6 kB/s eta 0:00:05
   --- ------------------------------------ 0.3/3.0 MB 1.0 MB/s eta 0:00:03
   ------- -------------------------------- 0.6/3.0 MB 1.8 MB/s eta 0:00:02
   ------------ --------------------------- 0.9/3.0 MB 2.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.9/3.0 MB 2.6 MB/s eta 0:00:01
   ------------------------ --------------- 1.8/3.0 

In [15]:
import h5py
print(h5py.__version__)
with h5py.File("test_file.h5", "w") as f:
    f.create_dataset("data", data=[1, 2, 3])

AttributeError: module 'h5py' has no attribute '__version__'